<a href="https://colab.research.google.com/github/gjrkdgh0772/Semi_project2/blob/main/zsemi2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# ! pip install -U finance-datareader
# ! pip install finance-datareader
# ! pip install pykrx
# ! pip install bokeh
# ! pip install yfinance
# ! pip install pandas_datareader

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from IPython.display import Image

from pykrx import stock
from pykrx import bond

from urllib.request import urlopen  
import bs4
import re
from bs4 import BeautifulSoup as bs4
import warnings
warnings.filterwarnings(action='ignore')

#-------------------- 차트 관련 속성 (한글처리, 그리드) -----------
#plt.rc('font', family='NanumGothicOTF') # For MacOS
plt.rcParams['font.family']= 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False
sns.set()

#-------------------- 주피터 , 출력결과 넓이 늘리기 ---------------
from IPython.core.display import display, HTML
display(HTML("<style>.container{width:100% !important;}</style>"))
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)
pd.set_option('max_colwidth', None)

plt.rc('font', family='NanumBarunGothic')

# Z-Score
----------------------
    * Z-Score 계산법.
        Z-Score = 1.2X1 + 1.4X2 + 3.3X3 + 0.6X4 + 0.999X5
----------------------
    * 변수설명
        X1  = (유동자산-유동부채)/자산총계  = 운전자본비율
        X2  = 이익잉여금/자산총계
        X3  = ROA
        X4  = 자기자본/부채총계
        X5  = S/A    매출액/자산총계
-----------------------

In [ ]:
df_null = pd.read_csv('/content/drive/MyDrive/금융/zscore파일/널값제거전전체데이터.csv',thousands=',')
df = pd.read_csv('/content/drive/MyDrive/금융/zscore파일/널값제거후전체데이터.csv',thousands=',')

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 621 entries, 0 to 620
Data columns (total 31 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  621 non-null    int64  
 1   종목_list     621 non-null    object 
 2   2018_매출액    621 non-null    int64  
 3   2019_매출액    621 non-null    object 
 4   2020_매출액    621 non-null    int64  
 5   2021_매출액    621 non-null    int64  
 6   2018_유동자산   621 non-null    int64  
 7   2019_유동자산   621 non-null    int64  
 8   2020_유동자산   621 non-null    int64  
 9   2021_유동자산   621 non-null    int64  
 10  2018_유동부채   621 non-null    object 
 11  2019_유동부채   621 non-null    object 
 12  2020_유동부채   621 non-null    object 
 13  2021_유동부채   621 non-null    object 
 14  2018_총자산    621 non-null    int64  
 15  2019_총자산    621 non-null    int64  
 16  2020_총자산    621 non-null    int64  
 17  2021_총자산    621 non-null    int64  
 18  2018_총부채    621 non-null    object 
 19  2019_총부채    621 non-null    o

In [ ]:
df.head()

,Unnamed: 0,종목_list,2018_매출액,2019_매출액,2020_매출액,2021_매출액,2018_유동자산,2019_유동자산,2020_유동자산,2021_유동자산,2018_유동부채,2019_유동부채,2020_유동부채,2021_유동부채,2018_총자산,2019_총자산,2020_총자산,2021_총자산,2018_총부채,2019_총부채,2020_총부채,2021_총부채,2017_roa,2018_roa,2019_roa,2020_roa,2021_roa,2018_이익잉여금,2019_이익잉여금,2020_이익잉여금,2021_이익잉여금
0,0,AJ네트웍스,10455,"10,003",8720,10267,15121,5011,3547,2739,"16,091","7,968","7,089","5,252",25751,18033,15882,13550,"20,833","14,559","13,002","9,925",0.9,1.5,1.9,-0.2,5.2,"1,572","1,988","1,806","2,479"
1,11,CJ제일제당,186701,"223,525",242457,262892,55224,79430,74699,77256,"66,804","82,022","70,827","79,159",194970,262806,256119,268558,"121,888","167,992","154,455","160,490",2.5,5.1,0.8,3.0,3.4,"36,943","37,864","44,068","49,402"
2,12,LG헬로비전,11780,"11,122",10579,10801,3339,3323,3760,4353,"3,545","5,829","4,224","2,682",19370,17907,14727,14881,"8,843","8,394","8,416","8,357",1.6,2.1,-5.1,-19.2,1.8,"6,690","5,676","2,474","2,686"
3,13,CS홀딩스,1226,"1,350",1316,1590,1885,1948,1899,1891,208,191,190,292,3044,3143,3237,3537,297,281,282,415,4.8,4.2,5.4,3.7,6.2,"1,699","1,797","1,872","2,007"
4,14,DB,2371,"2,385",2724,3269,837,839,886,845,605,592,"1,179",506,2884,3632,4725,6020,"1,508","1,522","1,588","1,642",-0.5,-1.6,1.4,1.5,4.0,436,481,540,772


### 2019_매출액 int처리위해 널값 제거

In [ ]:
df = df[df['2019_매출액']!='\xa0']

In [ ]:
df['2019_매출액'] = df['2019_매출액'].str.replace(',', '').astype('int64')

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 602 entries, 0 to 620
Data columns (total 31 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  602 non-null    int64  
 1   종목_list     602 non-null    object 
 2   2018_매출액    602 non-null    int64  
 3   2019_매출액    602 non-null    int64  
 4   2020_매출액    602 non-null    int64  
 5   2021_매출액    602 non-null    int64  
 6   2018_유동자산   602 non-null    int64  
 7   2019_유동자산   602 non-null    int64  
 8   2020_유동자산   602 non-null    int64  
 9   2021_유동자산   602 non-null    int64  
 10  2018_유동부채   602 non-null    int64  
 11  2019_유동부채   602 non-null    object 
 12  2020_유동부채   602 non-null    object 
 13  2021_유동부채   602 non-null    object 
 14  2018_총자산    602 non-null    int64  
 15  2019_총자산    602 non-null    int64  
 16  2020_총자산    602 non-null    int64  
 17  2021_총자산    602 non-null    int64  
 18  2018_총부채    602 non-null    object 
 19  2019_총부채    602 non-null    o

### 2018_유동부채 int처리위해 널값 제거

In [ ]:
df = df[df['2018_유동부채']!='\xa0']

In [ ]:
df['2018_유동부채'] = df['2018_유동부채'].str.replace(',', '').astype('int64')

### 2019_유동부채 int처리위해 널값 제거

In [ ]:
df = df[df['2019_유동부채']!='\xa0']

In [ ]:
df['2019_유동부채'] = df['2019_유동부채'].str.replace(',', '').astype('int64')

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 602 entries, 0 to 620
Data columns (total 31 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  602 non-null    int64  
 1   종목_list     602 non-null    object 
 2   2018_매출액    602 non-null    int64  
 3   2019_매출액    602 non-null    int64  
 4   2020_매출액    602 non-null    int64  
 5   2021_매출액    602 non-null    int64  
 6   2018_유동자산   602 non-null    int64  
 7   2019_유동자산   602 non-null    int64  
 8   2020_유동자산   602 non-null    int64  
 9   2021_유동자산   602 non-null    int64  
 10  2018_유동부채   602 non-null    int64  
 11  2019_유동부채   602 non-null    int64  
 12  2020_유동부채   602 non-null    object 
 13  2021_유동부채   602 non-null    object 
 14  2018_총자산    602 non-null    int64  
 15  2019_총자산    602 non-null    int64  
 16  2020_총자산    602 non-null    int64  
 17  2021_총자산    602 non-null    int64  
 18  2018_총부채    602 non-null    object 
 19  2019_총부채    602 non-null    o

### 2020_유동부채  int처리위해 널값 제거

In [ ]:
df = df[df['2020_유동부채']!='\xa0']

In [ ]:
df['2020_유동부채'] = df['2020_유동부채'].str.replace(',', '').astype('int64')

AttributeError: ignored

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 602 entries, 0 to 620
Data columns (total 31 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  602 non-null    int64  
 1   종목_list     602 non-null    object 
 2   2018_매출액    602 non-null    int64  
 3   2019_매출액    602 non-null    int64  
 4   2020_매출액    602 non-null    int64  
 5   2021_매출액    602 non-null    int64  
 6   2018_유동자산   602 non-null    int64  
 7   2019_유동자산   602 non-null    int64  
 8   2020_유동자산   602 non-null    int64  
 9   2021_유동자산   602 non-null    int64  
 10  2018_유동부채   602 non-null    int64  
 11  2019_유동부채   602 non-null    int64  
 12  2020_유동부채   602 non-null    int64  
 13  2021_유동부채   602 non-null    object 
 14  2018_총자산    602 non-null    int64  
 15  2019_총자산    602 non-null    int64  
 16  2020_총자산    602 non-null    int64  
 17  2021_총자산    602 non-null    int64  
 18  2018_총부채    602 non-null    object 
 19  2019_총부채    602 non-null    o

### 2021_유동부채  int처리위해 널값 제거

In [ ]:
df = df[df['2021_유동부채']!='\xa0']

In [ ]:
df['2021_유동부채'] = df['2021_유동부채'].str.replace(',', '').astype('int64')

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 602 entries, 0 to 620
Data columns (total 31 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  602 non-null    int64  
 1   종목_list     602 non-null    object 
 2   2018_매출액    602 non-null    int64  
 3   2019_매출액    602 non-null    int64  
 4   2020_매출액    602 non-null    int64  
 5   2021_매출액    602 non-null    int64  
 6   2018_유동자산   602 non-null    int64  
 7   2019_유동자산   602 non-null    int64  
 8   2020_유동자산   602 non-null    int64  
 9   2021_유동자산   602 non-null    int64  
 10  2018_유동부채   602 non-null    int64  
 11  2019_유동부채   602 non-null    int64  
 12  2020_유동부채   602 non-null    int64  
 13  2021_유동부채   602 non-null    int64  
 14  2018_총자산    602 non-null    int64  
 15  2019_총자산    602 non-null    int64  
 16  2020_총자산    602 non-null    int64  
 17  2021_총자산    602 non-null    int64  
 18  2018_총부채    602 non-null    object 
 19  2019_총부채    602 non-null    o

### 나머지 인트처리

In [ ]:
df = df[df['2018_총부채']!='\xa0']
df = df[df['2019_총부채']!='\xa0']
df = df[df['2020_총부채']!='\xa0']
df = df[df['2021_총부채']!='\xa0']

In [ ]:
df['2018_총부채'] = df['2018_총부채'].str.replace(',', '').astype('int64')
df['2019_총부채'] = df['2019_총부채'].str.replace(',', '').astype('int64')
df['2020_총부채'] = df['2020_총부채'].str.replace(',', '').astype('int64')
df['2021_총부채'] = df['2021_총부채'].str.replace(',', '').astype('int64')

In [ ]:
df = df[df['2018_이익잉여금']!='\xa0']
df = df[df['2019_이익잉여금']!='\xa0']
df = df[df['2020_이익잉여금']!='\xa0']
df = df[df['2021_이익잉여금']!='\xa0']

In [ ]:
df['2018_이익잉여금'] = df['2018_이익잉여금'].str.replace(',', '').astype('int64')
df['2019_이익잉여금'] = df['2019_이익잉여금'].str.replace(',', '').astype('int64')
df['2020_이익잉여금'] = df['2020_이익잉여금'].str.replace(',', '').astype('int64')
df['2021_이익잉여금'] = df['2021_이익잉여금'].str.replace(',', '').astype('int64')

In [ ]:
df =df[df['2017_roa']!='\xa0']

In [ ]:
df['2017_roa'] = df['2017_roa'].str.replace(',', '').astype('float')

In [ ]:
df = df[df['2017_roa'].isna() == False]

In [ ]:
df.head()

,Unnamed: 0,종목_list,2018_매출액,2019_매출액,2020_매출액,2021_매출액,2018_유동자산,2019_유동자산,2020_유동자산,2021_유동자산,2018_유동부채,2019_유동부채,2020_유동부채,2021_유동부채,2018_총자산,2019_총자산,2020_총자산,2021_총자산,2018_총부채,2019_총부채,2020_총부채,2021_총부채,2017_roa,2018_roa,2019_roa,2020_roa,2021_roa,2018_이익잉여금,2019_이익잉여금,2020_이익잉여금,2021_이익잉여금
0,0,AJ네트웍스,10455,10003,8720,10267,15121,5011,3547,2739,16091,7968,7089,5252,25751,18033,15882,13550,20833,14559,13002,9925,0.9,1.5,1.9,-0.2,5.2,1572,1988,1806,2479
1,11,CJ제일제당,186701,223525,242457,262892,55224,79430,74699,77256,66804,82022,70827,79159,194970,262806,256119,268558,121888,167992,154455,160490,2.5,5.1,0.8,3.0,3.4,36943,37864,44068,49402
2,12,LG헬로비전,11780,11122,10579,10801,3339,3323,3760,4353,3545,5829,4224,2682,19370,17907,14727,14881,8843,8394,8416,8357,1.6,2.1,-5.1,-19.2,1.8,6690,5676,2474,2686
3,13,CS홀딩스,1226,1350,1316,1590,1885,1948,1899,1891,208,191,190,292,3044,3143,3237,3537,297,281,282,415,4.8,4.2,5.4,3.7,6.2,1699,1797,1872,2007
4,14,DB,2371,2385,2724,3269,837,839,886,845,605,592,1179,506,2884,3632,4725,6020,1508,1522,1588,1642,-0.5,-1.6,1.4,1.5,4.0,436,481,540,772


## 자기자본 산출하기

In [ ]:
df['2018_자기자본'] = df['2018_총자산']-df['2018_총부채']
df['2019_자기자본'] = df['2019_총자산']-df['2019_총부채']
df['2020_자기자본'] = df['2020_총자산']-df['2020_총부채']
df['2021_자기자본'] = df['2021_총자산']-df['2021_총부채']

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 590 entries, 0 to 620
Data columns (total 35 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  590 non-null    int64  
 1   종목_list     590 non-null    object 
 2   2018_매출액    590 non-null    int64  
 3   2019_매출액    590 non-null    int64  
 4   2020_매출액    590 non-null    int64  
 5   2021_매출액    590 non-null    int64  
 6   2018_유동자산   590 non-null    int64  
 7   2019_유동자산   590 non-null    int64  
 8   2020_유동자산   590 non-null    int64  
 9   2021_유동자산   590 non-null    int64  
 10  2018_유동부채   590 non-null    int64  
 11  2019_유동부채   590 non-null    int64  
 12  2020_유동부채   590 non-null    int64  
 13  2021_유동부채   590 non-null    int64  
 14  2018_총자산    590 non-null    int64  
 15  2019_총자산    590 non-null    int64  
 16  2020_총자산    590 non-null    int64  
 17  2021_총자산    590 non-null    int64  
 18  2018_총부채    590 non-null    int64  
 19  2019_총부채    590 non-null    i

In [ ]:
df.to_csv('최종코스피목록',index=True)

In [ ]:
df.head()

,Unnamed: 0,종목_list,2018_매출액,2019_매출액,2020_매출액,2021_매출액,2018_유동자산,2019_유동자산,2020_유동자산,2021_유동자산,2018_유동부채,2019_유동부채,2020_유동부채,2021_유동부채,2018_총자산,2019_총자산,2020_총자산,2021_총자산,2018_총부채,2019_총부채,2020_총부채,2021_총부채,2017_roa,2018_roa,2019_roa,2020_roa,2021_roa,2018_이익잉여금,2019_이익잉여금,2020_이익잉여금,2021_이익잉여금,2018_자기자본,2019_자기자본,2020_자기자본,2021_자기자본
0,0,AJ네트웍스,10455,10003,8720,10267,15121,5011,3547,2739,16091,7968,7089,5252,25751,18033,15882,13550,20833,14559,13002,9925,0.9,1.5,1.9,-0.2,5.2,1572,1988,1806,2479,4918,3474,2880,3625
1,11,CJ제일제당,186701,223525,242457,262892,55224,79430,74699,77256,66804,82022,70827,79159,194970,262806,256119,268558,121888,167992,154455,160490,2.5,5.1,0.8,3.0,3.4,36943,37864,44068,49402,73082,94814,101664,108068
2,12,LG헬로비전,11780,11122,10579,10801,3339,3323,3760,4353,3545,5829,4224,2682,19370,17907,14727,14881,8843,8394,8416,8357,1.6,2.1,-5.1,-19.2,1.8,6690,5676,2474,2686,10527,9513,6311,6524
3,13,CS홀딩스,1226,1350,1316,1590,1885,1948,1899,1891,208,191,190,292,3044,3143,3237,3537,297,281,282,415,4.8,4.2,5.4,3.7,6.2,1699,1797,1872,2007,2747,2862,2955,3122
4,14,DB,2371,2385,2724,3269,837,839,886,845,605,592,1179,506,2884,3632,4725,6020,1508,1522,1588,1642,-0.5,-1.6,1.4,1.5,4.0,436,481,540,772,1376,2110,3137,4378


In [ ]:
df.columns

Index(['Unnamed: 0', '종목_list', '2018_매출액', '2019_매출액', '2020_매출액', '2021_매출액',
       '2018_유동자산', '2019_유동자산', '2020_유동자산', '2021_유동자산', '2018_유동부채',
       '2019_유동부채', '2020_유동부채', '2021_유동부채', '2018_총자산', '2019_총자산',
       '2020_총자산', '2021_총자산', '2018_총부채', '2019_총부채', '2020_총부채', '2021_총부채',
       '2017_roa', '2018_roa', '2019_roa', '2020_roa', '2021_roa',
       '2018_이익잉여금', '2019_이익잉여금', '2020_이익잉여금', '2021_이익잉여금', '2018_자기자본',
       '2019_자기자본', '2020_자기자본', '2021_자기자본'],
      dtype='object')

In [ ]:
df_2018 = df[['종목_list','2018_매출액','2018_유동자산', '2018_유동부채','2018_총자산','2018_총부채', '2018_roa','2018_이익잉여금','2018_자기자본']]
df_2019 = df[['종목_list','2019_매출액','2019_유동자산', '2019_유동부채','2019_총자산','2019_총부채', '2019_roa','2019_이익잉여금','2019_자기자본']]
df_2020 = df[['종목_list','2020_매출액','2020_유동자산', '2020_유동부채','2020_총자산','2020_총부채', '2020_roa','2020_이익잉여금','2020_자기자본']]
df_2021 = df[['종목_list','2021_매출액','2021_유동자산', '2021_유동부채','2021_총자산','2021_총부채', '2021_roa','2021_이익잉여금','2021_자기자본']]

In [ ]:
df_2018.columns = ['종목_list', '매출액', '유동자산', '유동부채', '자산총계', '부채총계', 'ROA', '이익잉여금', '자기자본']
df_2019.columns = ['종목_list', '매출액', '유동자산', '유동부채', '자산총계', '부채총계', 'ROA', '이익잉여금', '자기자본']
df_2020.columns = ['종목_list', '매출액', '유동자산', '유동부채', '자산총계', '부채총계', 'ROA', '이익잉여금', '자기자본']
df_2021.columns = ['종목_list', '매출액', '유동자산', '유동부채', '자산총계', '부채총계', 'ROA', '이익잉여금', '자기자본']

In [ ]:
def altman_z_score(df):
    if(len(df) != df.dropna().shape[0]):
        return 'Nan값 존재.'
    df['x1'] = round((df['유동자산'] - df['유동부채']) / df['자산총계'],2)
    df['x2'] = round(df['이익잉여금'] / df['자산총계'],2)
    df['x3'] = df['ROA']
    df['x4'] = round(df['자기자본'] / df['부채총계'],2)
    df['x5'] = round(df['매출액'] / df['자산총계'],2)
    
    df['z_score'] = round((1.2 * df['x1']) + (1.4 * df['x2']) + (3.3 * df['x3']) + (0.6 * df['x4']) + (0.999 * df['x5']), 2)
    
    return df

In [ ]:
altman_z_score(df_2018)

,종목_list,매출액,유동자산,유동부채,자산총계,부채총계,ROA,이익잉여금,자기자본,x1,x2,x3,x4,x5,z_score
0,AJ네트웍스,10455,15121,16091,25751,20833,1.5,1572,4918,-0.04,0.06,1.5,0.24,0.41,5.54
1,CJ제일제당,186701,55224,66804,194970,121888,5.1,36943,73082,-0.06,0.19,5.1,0.60,0.96,18.34
2,LG헬로비전,11780,3339,3545,19370,8843,2.1,6690,10527,-0.01,0.35,2.1,1.19,0.61,8.73
3,CS홀딩스,1226,1885,208,3044,297,4.2,1699,2747,0.55,0.56,4.2,9.25,0.40,21.25
4,DB,2371,837,605,2884,1508,-1.6,436,1376,0.08,0.15,-1.6,0.91,0.82,-3.61
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
616,휠라홀딩스,29546,13379,6723,33043,17978,6.6,8224,15065,0.20,0.25,6.6,0.84,0.89,23.76
617,휴비스,12625,5166,4522,9581,5500,2.0,2103,4081,0.07,0.22,2.0,0.74,1.32,8.75
618,휴스틸,5888,3000,1778,6035,1974,2.7,2366,4061,0.20,0.39,2.7,2.06,0.98,11.91
619,TKG휴켐스,7629,5296,1007,8647,1987,12.0,4556,6660,0.50,0.53,12.0,3.35,0.88,43.83


In [ ]:
altman_z_score(df_2019)

,종목_list,매출액,유동자산,유동부채,자산총계,부채총계,ROA,이익잉여금,자기자본,x1,x2,x3,x4,x5,z_score
0,AJ네트웍스,10003,5011,7968,18033,14559,1.9,1988,3474,-0.16,0.11,1.9,0.24,0.55,6.93
1,CJ제일제당,223525,79430,82022,262806,167992,0.8,37864,94814,-0.01,0.14,0.8,0.56,0.85,4.01
2,LG헬로비전,11122,3323,5829,17907,8394,-5.1,5676,9513,-0.14,0.32,-5.1,1.13,0.62,-15.25
3,CS홀딩스,1350,1948,191,3143,281,5.4,1797,2862,0.56,0.57,5.4,10.19,0.43,25.83
4,DB,2385,839,592,3632,1522,1.4,481,2110,0.07,0.13,1.4,1.39,0.66,6.38
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
616,휠라홀딩스,34504,15868,8674,37702,19693,9.6,10800,18009,0.19,0.29,9.6,0.91,0.92,33.78
617,휴비스,10293,2885,3422,8772,4810,0.0,1996,3962,-0.06,0.23,0.0,0.82,1.17,1.91
618,휴스틸,5370,2650,1462,5791,1825,-2.1,2193,3966,0.21,0.38,-2.1,2.17,0.93,-3.91
619,TKG휴켐스,6598,4268,1680,9139,2422,7.9,4779,6717,0.28,0.52,7.9,2.77,0.72,29.52


In [ ]:
altman_z_score(df_2020)

,종목_list,매출액,유동자산,유동부채,자산총계,부채총계,ROA,이익잉여금,자기자본,x1,x2,x3,x4,x5,z_score
0,AJ네트웍스,8720,3547,7089,15882,13002,-0.2,1806,2880,-0.22,0.11,-0.2,0.22,0.55,-0.09
1,CJ제일제당,242457,74699,70827,256119,154455,3.0,44068,101664,0.02,0.17,3.0,0.66,0.95,11.51
2,LG헬로비전,10579,3760,4224,14727,8416,-19.2,2474,6311,-0.03,0.17,-19.2,0.75,0.72,-61.99
3,CS홀딩스,1316,1899,190,3237,282,3.7,1872,2955,0.53,0.58,3.7,10.48,0.41,20.36
4,DB,2724,886,1179,4725,1588,1.5,540,3137,-0.06,0.11,1.5,1.98,0.58,6.80
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
616,휠라홀딩스,31288,16861,9955,37584,19011,5.3,11793,18573,0.18,0.31,5.3,0.98,0.83,19.56
617,휴비스,9203,3979,2980,8729,3971,9.4,2762,4758,0.11,0.32,9.4,1.20,1.05,33.37
618,휴스틸,3656,2615,1248,6041,1897,1.5,2278,4144,0.23,0.38,1.5,2.18,0.61,7.68
619,TKG휴켐스,5935,4345,908,8666,1794,5.6,4931,6872,0.40,0.57,5.6,3.83,0.68,22.74


In [ ]:
altman_z_score(df_2021)

,종목_list,매출액,유동자산,유동부채,자산총계,부채총계,ROA,이익잉여금,자기자본,x1,x2,x3,x4,x5,z_score
0,AJ네트웍스,10267,2739,5252,13550,9925,5.2,2479,3625,-0.19,0.18,5.2,0.37,0.76,18.17
1,CJ제일제당,262892,77256,79159,268558,160490,3.4,49402,108068,-0.01,0.18,3.4,0.67,0.98,12.84
2,LG헬로비전,10801,4353,2682,14881,8357,1.8,2686,6524,0.11,0.18,1.8,0.78,0.73,7.52
3,CS홀딩스,1590,1891,292,3537,415,6.2,2007,3122,0.45,0.57,6.2,7.52,0.45,26.76
4,DB,3269,845,506,6020,1642,4.0,772,4378,0.06,0.13,4.0,2.67,0.54,15.60
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
616,휠라홀딩스,37940,20411,10514,42889,19824,8.4,14060,23065,0.23,0.33,8.4,1.16,0.88,30.03
617,휴비스,10826,3419,3313,8434,3756,0.4,2652,4678,0.01,0.31,0.4,1.25,1.28,3.79
618,휴스틸,6165,4119,1756,9519,3255,4.9,2647,6264,0.25,0.28,4.9,1.92,0.65,18.66
619,TKG휴켐스,8612,5205,1832,9493,2297,8.5,5286,7196,0.36,0.56,8.5,3.13,0.91,32.05


In [ ]:
df_2018.to_csv('2018년',index=True)
df_2019.to_csv('2019년',index=True)
df_2020.to_csv('2020년',index=True)
df_2021.to_csv('2021년',index=True)

# 업종별 Columns 추가.

In [ ]:
for ticker in stock.get_index_ticker_list():
    print(ticker, stock.get_index_ticker_name(ticker))

1001 코스피
1002 코스피 대형주
1003 코스피 중형주
1004 코스피 소형주
1005 음식료품
1006 섬유의복
1007 종이목재
1008 화학
1009 의약품
1010 비금속광물
1011 철강금속
1012 기계
1013 전기전자
1014 의료정밀
1015 운수장비
1016 유통업
1017 전기가스업
1018 건설업
1019 운수창고업
1020 통신업
1021 금융업
1022 은행
1024 증권
1025 보험
1026 서비스업
1027 제조업
1028 코스피 200
1034 코스피 100
1035 코스피 50
1150 코스피 200 커뮤니케이션서비스
1151 코스피 200 건설
1152 코스피 200 중공업
1153 코스피 200 철강/소재
1154 코스피 200 에너지/화학
1155 코스피 200 정보기술
1156 코스피 200 금융
1157 코스피 200 생활소비재
1158 코스피 200 경기소비재
1159 코스피 200 산업재
1160 코스피 200 헬스케어
1167 코스피 200 중소형주
1182 코스피 200 초대형제외 지수
1224 코스피 200 비중상한 30%
1227 코스피 200 비중상한 25%
1232 코스피 200 비중상한 20%
1244 코스피200제외 코스피지수
1894 코스피 200 TOP 10


In [ ]:
df = stock.get_market_price_change("20180101", "20211231")

In [ ]:
del_list = df[df['등락률'] == -100].index.tolist()

In [ ]:
df = df.drop(del_list,axis=0)

In [ ]:
crawling_index = df.index.tolist()

In [ ]:
crawling_index

['095570',
 '006840',
 '027410',
 '282330',
 '138930',
 '001460',
 '001465',
 '001040',
 '079160',
 '000120',
 '011150',
 '011155',
 '001045',
 '097950',
 '097955',
 '000590',
 '012030',
 '016610',
 '005830',
 '000990',
 '000995',
 '139130',
 '001530',
 '000210',
 '001880',
 '000215',
 '004840',
 '155660',
 '069730',
 '017940',
 '007700',
 '114090',
 '078930',
 '006360',
 '001250',
 '007070',
 '078935',
 '012630',
 '039570',
 '089470',
 '011200',
 '082740',
 '003560',
 '175330',
 '234080',
 '001060',
 '001067',
 '001065',
 '096760',
 '105560',
 '002380',
 '009070',
 '009440',
 '119650',
 '092220',
 '016380',
 '016385',
 '001390',
 '033180',
 '001940',
 '025000',
 '092230',
 '000040',
 '044450',
 '030200',
 '033780',
 '030210',
 '058850',
 '058860',
 '093050',
 '003550',
 '034220',
 '051900',
 '051905',
 '003555',
 '032640',
 '011070',
 '066570',
 '066575',
 '037560',
 '051910',
 '051915',
 '079550',
 '006260',
 '010120',
 '000680',
 '229640',
 '001120',
 '108670',
 '108675',
 '023150',

## 재무건전성 기반으로 파생변수 크롤링

In [ ]:
kospi_list2018 = stock.get_market_ticker_list("20180101", market="KOSPI")
len(kospi_list2018)

887

In [ ]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
from tqdm import tqdm
from time import sleep
import urllib.request
당좌비율_totlist = []
부채비율_totlist = []
이자보상비율_totlist = []
종목_list = []
for index in tqdm(kospi_list2018, desc='tqdm test processing'):

    try:
        

        url = "https://comp.fnguide.com/SVO2/ASP/SVD_FinanceRatio.asp?pGB=1&gicode=A"+index+"&cID=&MenuYn=Y&ReportGB=&NewMenuID=104&stkGb=701"
        # 웹사이트 정보 요청
        page = urllib.request.urlopen(url)

        # 해당 페이지는 cp949 방식의 인코딩 사용
        html = page.read().decode('utf8')

        # html.parser로 html 형식 태그에서 데이터 추출 준비
        soup = BeautifulSoup(html, 'html.parser')

        당좌비율 =  soup.find(id='compBody').find_all('tr')[5].find_all('td')
        당좌비율_list = []
        for number in 당좌비율:
            당좌비율_list.append(number.string)
        당좌비율_totlist.append(당좌비율_list)

        
        부채비율 =  soup.find(id='compBody').find_all('tr')[8].find_all('td')
        부채비율_list = []
        for number in 부채비율:
            부채비율_list.append(number.string)
        부채비율_totlist.append(부채비율_list)

        이자보상비율 =  soup.find(id='compBody').find_all('tr')[17].find_all('td')
        이자보상비율_list = []
        for number in 이자보상비율:
            이자보상비율_list.append(number.string)
        이자보상비율_totlist.append(이자보상비율_list)
        
        
        
        종목이름 = soup.select_one('#giName').text
        종목_list.append(종목이름)
    except:
        pass
print(종목_list)
print(len(당좌비율_totlist),len(종목_list),len(부채비율_totlist),len(이자보상비율_totlist))
# for i in tmp:
#     print(i.string)




tqdm test processing: 100%|██████████| 887/887 [36:01<00:00,  2.44s/it]

['AJ네트웍스', 'SK렌터카', 'AK홀딩스', 'BGF', 'BGF리테일', 'BNK금융지주', 'BYC', 'CJ', 'CJ\xa0CGV', 'CJ대한통운', 'CJ씨푸드', 'CJ제일제당', 'LG헬로비전', 'CS홀딩스', 'DB', 'DB금융투자', 'DB손해보험', 'DB하이텍', 'DGB금융지주', 'DRB동일', 'DSR', 'DSR제강', 'E1', 'F&F홀딩스', 'GKL', 'GS', 'GS건설', 'GS글로벌', 'GS리테일', 'IHQ', 'JB금융지주', 'JW생명과학', 'JW중외제약', 'JW홀딩스', 'KB금융', 'KCC', 'KCTC', 'KC그린홀딩스', 'KC코트렐', 'KEC', '컨버즈', 'KG케미칼', 'KISCO홀딩스', '그린케미칼', 'KPX케미칼', 'KPX홀딩스', 'KR모터스', 'KSS해운', 'KT', 'KT&G', '다올투자증권', 'KTcs', 'LF', 'LG', 'LG디스플레이', 'LX인터내셔널', 'LG생활건강', 'LG유플러스', 'LG이노텍', 'LG전자', 'LX하우시스', 'LG화학', 'LIG넥스원', 'LS', 'LS네트웍스', 'LS\xa0ELECTRIC', 'LS전선아시아', 'MH에탄올', 'NAVER', 'NHN', 'NH투자증권', 'NICE', 'NI스틸', 'NPC', 'OCI', 'POSCO홀딩스', 'SNT에너지', 'SNT모티브', 'SNT중공업', 'SNT홀딩스', 'S-Oil', 'SBS', 'SBS미디어홀딩스', 'SG세계물산', 'SG글로벌', 'SH에너지화학', 'SIMPAC', 'SJM', 'SJM홀딩스', 'SK', 'SKC', 'SK가스', 'SK네트웍스', 'SK디앤디', 'SK이노베이션', 'SK증권', 'SK디스커버리', 'SK텔레콤', 'SK하이닉스', 'SPC삼립', 'STX', 'STX엔진', 'STX중공업', 'TBH글로벌', 'TCC스틸', 'WISCOM', 'YG\xa0PLUS', '가온전선', '강남제비스코', '강원랜드', 

In [ ]:
# 재무건전성요소.columns = [['종목','2018_당좌비율','20119_당좌비율','2020_당좌비율','2021_당좌비율','2018_부채비율','2019_부채비율','2020_부채비율','2021_부채비율'
# ,'2018_이자보상비율','2019_이자보상비율','2020_이자보상비율','2021_이자보상비율']]

ValueError: ignored

In [ ]:
재무건전성요소 = pd.DataFrame(columns = ['종목','2018_당좌비율','2019_당좌비율','2020_당좌비율','2021_당좌비율','2018_부채비율','2019_부채비율','2020_부채비율','2021_부채비율'
,'2018_이자보상비율','2019_이자보상비율','2020_이자보상비율','2021_이자보상비율'])

In [ ]:
재무건전성요소['종목'] = 종목_list 

In [ ]:
재무건전성요소

,종목,2018_당좌비율,2019_당좌비율,2020_당좌비율,2021_당좌비율,2018_부채비율,2019_부채비율,2020_부채비율,2021_부채비율,2018_이자보상비율,2019_이자보상비율,2020_이자보상비율,2021_이자보상비율
0,AJ네트웍스,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,SK렌터카,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,AK홀딩스,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,BGF,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,BGF리테일,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
766,휴비스,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
767,휴스틸,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
768,TKG휴켐스,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
769,흥국화재,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
당좌비율_totlist[:10]

[['21.9', '90.8', '55.5', '45.3', '48.4'],
 ['13.0', '14.9', '32.0', '27.0', '31.6'],
 ['76.5', '81.1', '68.6', '58.8', '52.2'],
 ['489.3', '540.9', '497.0', '667.6', '365.9'],
 ['69.3', '77.6', '68.7', '70.5', '68.9'],
 [],
 ['37.1', '42.9', '66.4', '67.4', '78.4'],
 ['69.8', '61.2', '69.0', '76.4', '72.7'],
 ['50.6', '55.5', '58.5', '71.2', '62.3'],
 ['94.6', '82.6', '91.7', '89.5', '94.9']]

In [ ]:
for i in range(len(당좌비율_totlist)):
    try:
                
        재무건전성요소['2018_당좌비율'][i]=당좌비율_totlist[i][1]
        재무건전성요소['2019_당좌비율'][i]=당좌비율_totlist[i][2]
        재무건전성요소['2020_당좌비율'][i]=당좌비율_totlist[i][3]
        재무건전성요소['2021_당좌비율'][i]=당좌비율_totlist[i][4]
        
        
        재무건전성요소['2018_부채비율'][i]=부채비율_totlist[i][1]
        재무건전성요소['2019_부채비율'][i]=부채비율_totlist[i][2]
        재무건전성요소['2020_부채비율'][i]=부채비율_totlist[i][3]
        재무건전성요소['2021_부채비율'][i]=부채비율_totlist[i][4]

            
        재무건전성요소['2018_이자보상비율'][i]=이자보상비율_totlist[i][1]
        재무건전성요소['2019_이자보상비율'][i]=이자보상비율_totlist[i][2]
        재무건전성요소['2020_이자보상비율'][i]=이자보상비율_totlist[i][3]
        재무건전성요소['2021_이자보상비율'][i]=이자보상비율_totlist[i][4]

    except:
        pass
    

In [ ]:
재무건전성요소.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 771 entries, 0 to 770
Data columns (total 13 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   종목           771 non-null    object
 1   2018_당좌비율    753 non-null    object
 2   2019_당좌비율    753 non-null    object
 3   2020_당좌비율    753 non-null    object
 4   2021_당좌비율    753 non-null    object
 5   2018_부채비율    730 non-null    object
 6   2019_부채비율    730 non-null    object
 7   2020_부채비율    730 non-null    object
 8   2021_부채비율    730 non-null    object
 9   2018_이자보상비율  730 non-null    object
 10  2019_이자보상비율  730 non-null    object
 11  2020_이자보상비율  730 non-null    object
 12  2021_이자보상비율  730 non-null    object
dtypes: object(13)
memory usage: 78.4+ KB


In [ ]:
재무건전성요소

,종목,2018_당좌비율,2019_당좌비율,2020_당좌비율,2021_당좌비율,2018_부채비율,2019_부채비율,2020_부채비율,2021_부채비율,2018_이자보상비율,2019_이자보상비율,2020_이자보상비율,2021_이자보상비율
0,AJ네트웍스,90.8,55.5,45.3,48.4,423.6,419.2,451.4,273.8,,0.4,0.6,1.4
1,SK렌터카,14.9,32.0,27.0,31.6,375.1,377.2,381.0,491.4,0.8,1.5,1.9,1.9
2,AK홀딩스,81.1,68.6,58.8,52.2,130.5,202.7,228.8,278.3,9.8,1.8,,
3,BGF,540.9,497.0,667.6,365.9,7.2,8.5,7.9,14.4,23.4,78.0,42.2,6.8
4,BGF리테일,77.6,68.7,70.5,68.9,184.2,248.9,240.3,220.8,27.7,10.3,9.0,10.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...
766,휴비스,74.8,43.8,94.8,56.0,134.8,121.4,83.5,80.3,4.5,0.5,4.7,1.3
767,휴스틸,78.5,89.9,109.2,100.2,48.6,46.0,45.8,52.0,3.0,,6.1,24.3
768,TKG휴켐스,495.3,237.1,447.9,257.2,29.8,36.1,26.1,31.9,38.0,31.8,40.5,49.8
769,흥국화재,5.7,5.6,5.5,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
컬럼=재무건전성요소.columns.tolist()

In [ ]:
for name in 컬럼:
    
    재무건전성요소 = 재무건전성요소[재무건전성요소[name]!='\xa0']


In [ ]:
재무건전성요소

,종목,2018_당좌비율,2019_당좌비율,2020_당좌비율,2021_당좌비율,2018_부채비율,2019_부채비율,2020_부채비율,2021_부채비율,2018_이자보상비율,2019_이자보상비율,2020_이자보상비율,2021_이자보상비율
1,SK렌터카,14.9,32.0,27.0,31.6,375.1,377.2,381.0,491.4,0.8,1.5,1.9,1.9
3,BGF,540.9,497.0,667.6,365.9,7.2,8.5,7.9,14.4,23.4,78.0,42.2,6.8
4,BGF리테일,77.6,68.7,70.5,68.9,184.2,248.9,240.3,220.8,27.7,10.3,9.0,10.7
5,BNK금융지주,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,BYC,42.9,66.4,67.4,78.4,81.3,59.8,50.0,42.1,2.9,2.9,4.5,9.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...
764,휠라홀딩스,105.0,98.2,107.5,122.1,119.3,109.4,102.4,86.0,6.6,8.4,9.4,21.4
765,휴니드,64.3,82.6,69.3,75.7,109.3,125.3,106.7,100.7,94.9,41.0,58.0,77.3
766,휴비스,74.8,43.8,94.8,56.0,134.8,121.4,83.5,80.3,4.5,0.5,4.7,1.3
768,TKG휴켐스,495.3,237.1,447.9,257.2,29.8,36.1,26.1,31.9,38.0,31.8,40.5,49.8


In [ ]:
재무건전성요소 = 재무건전성요소[재무건전성요소['2021_이자보상비율'].isna()==False]

In [ ]:
재무건전성요소.to_csv('재무건전성요소',index=True)

In [ ]:
재무건전성요소 = pd.read_csv('/content/재무건전성요소',thousands=',')

In [ ]:
컬=['2018_당좌비율',
 '2019_당좌비율',
 '2020_당좌비율',
 '2021_당좌비율',
 '2018_부채비율',
 '2019_부채비율',
 '2020_부채비율',
 '2021_부채비율',
 '2018_이자보상비율',
 '2019_이자보상비율',
 '2020_이자보상비율',
 '2021_이자보상비율']

In [ ]:
for columns in 컬:

    재무건전성요소[columns].astype(float)

In [ ]:
재무건전성요소.to_csv('재무건전성',index=True)

In [ ]:
재무건전성요소

,Unnamed: 0,종목,2018_당좌비율,2019_당좌비율,2020_당좌비율,2021_당좌비율,2018_부채비율,2019_부채비율,2020_부채비율,2021_부채비율,2018_이자보상비율,2019_이자보상비율,2020_이자보상비율,2021_이자보상비율
0,1,SK렌터카,14.9,32.0,27.0,31.6,375.1,377.2,381.0,491.4,0.8,1.5,1.9,1.9
1,3,BGF,540.9,497.0,667.6,365.9,7.2,8.5,7.9,14.4,23.4,78.0,42.2,6.8
2,4,BGF리테일,77.6,68.7,70.5,68.9,184.2,248.9,240.3,220.8,27.7,10.3,9.0,10.7
3,6,BYC,42.9,66.4,67.4,78.4,81.3,59.8,50.0,42.1,2.9,2.9,4.5,9.4
4,7,CJ,61.2,69.0,76.4,72.7,153.4,176.3,160.0,154.8,3.6,2.3,2.4,3.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
441,763,후성,118.3,96.8,70.8,74.2,79.8,117.7,120.3,117.4,19.6,3.3,0.5,6.5
442,764,휠라홀딩스,105.0,98.2,107.5,122.1,119.3,109.4,102.4,86.0,6.6,8.4,9.4,21.4
443,765,휴니드,64.3,82.6,69.3,75.7,109.3,125.3,106.7,100.7,94.9,41.0,58.0,77.3
444,766,휴비스,74.8,43.8,94.8,56.0,134.8,121.4,83.5,80.3,4.5,0.5,4.7,1.3


# ref:https://m.blog.naver.com/PostView.naver?isHttpsRedirect=true&blogId=finata&logNo=221191125946